In [ ]:
!pip install requests PyMuPDF

In [4]:
!pip install ftfy

# Ollama Code aus Git

In [1]:
import requests
import json
import re
import ftfy

class OllamaApi:

    HOST = "https://f2ki-h100-1.f2.htw-berlin.de"
    PORT = 11435

    TIMEOUT = 120
    STREAM_RESPONSE = False # Only for debug purposes. !! Result will be null !!

    THINKING = False

    FALSE_RETURN = {"result": None, "time": 0, "token": 0, "info": {}}

    DEFAULT_OPTIONS = {
        "num_ctx": 2048,        # Default: 2048
        "repeat_last_n": 64,    # Default: 64, 0 = disabled, -1 = num_ctx
        "repeat_penalty": 1.1,  # Default: 1.1
        "temperature": 0.8,     # Default: 0.8
        "seed": 0,              # Default: 0
        "stop": [],             # No default
        "num_predict": -1,      # Default: -1, infinite generation
        "top_k": 40,            # Default: 40
        "top_p": 0.9,           # Default: 0.9
        "min_p": 0.0            # Default: 0.0
    }

    @staticmethod
    def fix_invalid_escapes(s):
        if not isinstance(s, str):
            return s

        try:
            s_re = ftfy.fix_text(s)
            if s_re != s:
                s = s_re
        except Exception as e:
            print(f"Encoding failed for text: {e}")

        return s

    @classmethod
    def models(cls):
        url = f"{cls.HOST}:{cls.PORT}/api/tags"
        headers = {
            "accept": "application/json",
        }
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Request failed with status {response.status_code}: {response.text}")
            return False
        else:
            try:
                json_data = response.json()
                return json_data.get("models", [])

            except json.JSONDecodeError as e:
                print(f"Failed to parse JSON: {e}")
                return False

    @classmethod
    def pull_model(cls, name:str, tag:str):
        url = f"{cls.HOST}:{cls.PORT}/api/pull"
        headers = {
            "Content-Type": "application/json",
            "accept": "application/json"
        }
        payload = {
            "model": f"{name}:{tag}"
        }
        try:
            response = requests.post(url, headers=headers, json=payload, stream=True, timeout=cls.TIMEOUT)
            if response.status_code != 200:
                print(f"Pull request failed with status {response.status_code}: {response.text}")
                return False

            for line in response.iter_lines():
                if line:
                    try:
                        progress = json.loads(line)
                        if 'status' in progress:
                            print(f"Pulling model: {progress['status']}")
                        if progress['status'] == "success":
                            return True
                    except json.JSONDecodeError:
                        continue

            return True
        except Exception as e:
            print(f"Failed to pull model: {e}")
            return False

    @classmethod
    def completion(cls, prompt:str, model="phi4:latest", schema=None, options=None):
        payload = {
            "model": model,
            "prompt" : prompt,
            "options": {
                **cls.DEFAULT_OPTIONS,
                **options
            } if options is not None else cls.DEFAULT_OPTIONS,
        }
        if schema is not None:
            payload["format"] = schema

        return cls.api_request(payload, force_json=False if schema is None else True)

    @classmethod
    def chat(cls, chat, model="phi4:latest", schema=None, options=None):
        payload = {
            "model": model,
            "messages": chat,
            "options": {
                **cls.DEFAULT_OPTIONS,
                **options
            } if options is not None else cls.DEFAULT_OPTIONS,
        }
        if schema is not None:
            payload["format"] = schema

        return cls.api_request(payload, force_json=False if schema is None else True)

    @classmethod
    def api_request(cls, payload, force_json:bool):
        if "messages" in payload:
            # Chat Request
            url = f"{cls.HOST}:{cls.PORT}/api/chat"
        else:
            # Completion Request
            url = f"{cls.HOST}:{cls.PORT}/api/generate"

        headers = {
            "Content-Type": "application/json",
            "accept": "application/json"
        }

        payload = {
            **payload,
            "think": cls.THINKING,
            "stream": cls.STREAM_RESPONSE,
            "keep_alive": "5m"
        }

        try:
            response = requests.post(url, headers=headers, json=payload, stream=cls.STREAM_RESPONSE, timeout=cls.TIMEOUT)

            if cls.STREAM_RESPONSE:
                for line in response.iter_lines(decode_unicode=True):
                    try:
                        chunk = json.loads(line)

                        if 'message' in chunk and 'content' in chunk['message']:
                            content = chunk['message']['content']
                            print(content, end='', flush=True)

                        if 'done' in chunk and chunk['done']:
                            break  # Exit the loop if done is True
                    except json.JSONDecodeError as e:
                        print(f"ERROR: Failed to decode JSON during streaming: {e}")
                return {**cls.FALSE_RETURN, "info": {"error": 'Streaming mode does not retreive a value'}}
            else:
                return cls.secure_json_response(response) if force_json else cls.secure_text_response(response)

        except requests.exceptions.Timeout:
            print(f"ERROR: The request took to long. Adjust the timeout ({cls.TIMEOUT}) as needed")
            return {**cls.FALSE_RETURN, "info": {"error": f"Request timeout ({cls.TIMEOUT}) reached"}}
        except Exception as e:
            print(f"ERROR: Request exception: {e}")
            return {**cls.FALSE_RETURN, "info": {"error": f"Request exception: {e}"}}


    @classmethod
    def secure_json_response(cls, response):

        text_response = cls.secure_text_response(response)

        if text_response.get("result") is None:
            return text_response

        message = str(text_response.get("result"))

        markdown_response = False
        thinking_block = False

        if message.strip().startswith("<think>"):
            print('WARN: Model returned <think> reasoning block before JSON')
            message = re.sub(r"^\s*<think>.*?</think>\s*", "", message, flags=re.DOTALL).strip()
            thinking_block = True

        match = re.search(r'```json(.*?)```', message, re.DOTALL)
        if match:
            # Remove everything except the content in "```json" to "```"
            print('WARN: Model returned markdown instead of only JSON')
            message = match.group(1).strip()
            markdown_response = True

        try:
            # Try to parse To json
            result = json.loads(message)

            # Overwrite text result with json dict
            text_response["result"] = dict(result)
            text_response["info"] = {
                "thinking": thinking_block,
                "markdown": markdown_response
            }
            return text_response

        except json.JSONDecodeError as e:
            print(f"ERROR: Failed to decode JSON: {e}")
            return {**cls.FALSE_RETURN, "info": {"error": 'JSON decode error on the model\'s response'}}

        except Exception as e:
            print(f"ERROR: Failed to parse JSON: {e}")
            return {**cls.FALSE_RETURN, "info": {"error": str(e)}}

    @classmethod
    def secure_text_response(cls, response):

        try:
            parsed_json = response.json()

            if response.status_code != 200:
                err_msg = parsed_json.get('error', 'Unknown error')
                print(f"ERROR: Request failed with status {response.status_code}: {err_msg}")
                return {**cls.FALSE_RETURN, "info":{"error":err_msg}}

            if 'done' not in parsed_json or parsed_json.get('done') is False:
                print("ERROR: Response has returned but Model didn't complete the answer")
                return {**cls.FALSE_RETURN, "info": {"error": 'Incomplete answer'}}

            # LLM Chat return as string
            message = parsed_json.get('message').get('content') if "message" in parsed_json else parsed_json.get('response')

            microseconds_elapsed = parsed_json.get('total_duration')
            seconds_elapsed = round(microseconds_elapsed / 1000000000, 3)
            token_count = parsed_json.get('eval_count')

            return {
                "result": cls.fix_invalid_escapes(message),
                "time": float(seconds_elapsed),
                "token": int(token_count),
                "info": {}
            }

        except json.JSONDecodeError as e:
            print(f"ERROR: Failed to decode JSON: {e}")
            return {**cls.FALSE_RETURN, "info": {"error": 'JSON decode error on the ollama server\'s response'}}

        except Exception as e:
            print(f"ERROR: Failed to parse JSON: {e}")
            return {**cls.FALSE_RETURN, "info": {"error": str(e)}}


# Versprechen extrahieren

In [57]:
import os
import re
import fitz  # PyMuPDF

# Modell wählen
model_name = "llama3.1:70b" #Sehr leistungsfähig, gute Sprachverarbeitung auch auf Deutsch, Gute Wahl für präzise Informationsextraktion
print(f"-------\nUsing model: {model_name}")

# Ordnerpfad mit PDFs (anpassen)
pdf_folder_path = "./Berichte"

# Funktion zur Extraktion von Text aus einer PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        doc = fitz.open(pdf_path)
        for page in doc:
            text += page.get_text()
        doc.close()
    except Exception as e:
        print(f"Fehler beim Lesen von {pdf_path}: {e}")
    return text

# Funktion zum Chunken von Text basierend auf Satzgrenzen
def chunk_text_into_sentences(text, max_chunk_chars=3000):
    # Zerlege den Text an Satzzeichen + Leerzeichen (nach Punkt, Fragezeichen oder Ausrufezeichen)
    sentences = re.split(r'(?<=[.!?])\s+', text)
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) + 1 <= max_chunk_chars:
            current_chunk += " " + sentence
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

# Prompt-Vorlage
prompt_template = """
Du bist ein Textanalyse-Tool, das Nachhaltigkeitsversprechen aus Unternehmensdokumenten extrahiert.

Analysiere den folgenden Text und extrahiere **alle Nachhaltigkeitsversprechen** in **klaren, prägnanten Stichpunkten**.

**Wichtige Regeln:**
- Entferne alle doppelten oder sinngemäß gleichen Aussagen (Deduplizierung).
- Formuliere jeden Punkt neutral und sachlich.
- Vermeide Wiederholungen oder Variationen derselben Aussage.
- Achte auf konkrete Absichten, Maßnahmen, Ziele oder Prinzipien im Bereich Umwelt, Soziales oder Unternehmensführung (ESG).
- Gib nur die Versprechen wieder – keine allgemeinen Aussagen oder Beschreibungen.

**Format:**
- Gib die Ergebnisse in einer nummerierten Liste zurück.
- Max. ein Satz pro Stichpunkt.

**Text:**  

{text}
"""

# PDFs iterieren

print(f"\n→ Text wird analysiert...")
prompt = prompt_template.format(text=promise_vw)

try:
    result = OllamaApi.completion(prompt, model=model_name)
    extracted = result.get("result", "").strip()
    if extracted:
        print(f"\n### Extrahierte Nachhaltigkeitsversprechen:\n")
        print(extracted)

except Exception as e:
    print(f"Fehler bei der Modellabfrage")


-------
Using model: llama3.1:70b

→ Text wird analysiert...

### Extrahierte Nachhaltigkeitsversprechen:

Der vorliegende Text befasst sich mit den Nachhaltigkeitsbemühungen und -zielen des Volkswagen-Konzerns, insbesondere in Bezug auf die Lieferkette. Hier sind einige der wichtigsten Punkte und Ziele:

**Lieferantenmanagement**

* Ziel: 80% der direkten Lieferanten mit zertifiziertem Umweltmanagementsystem gemäß ISO-14001- oder EMAS-Validierung (2023)
* Ziel: 85% der direkten Lieferanten mit zertifiziertem Umweltmanagementsystem gemäß ISO-14001- oder EMAS-Validierung (2022)
* Anteil der neuen Lieferanten, die anhand von Sozial- und Umweltkriterien bewertet wurden: 26% (2023) bzw. 12% (2022)

**Nachhaltigkeitsanforderungen**

* Alle Lieferanten müssen unsere Nachhaltigkeitsanforderungen erfüllen.
* Lieferanten mit einem A- oder B-Rating erfüllen unsere Anforderungen in ausreichendem Umfang und sind somit vergabefähig.

**Rohstofflieferketten**

* Ziel: Transparenz in der Lieferkette 

# extrahierte Versprechen automatisiert überprüfen

In [26]:
!pip install playwright
!playwright install


133.5 MiB [                    ] 0% 0.0s133.5 MiB [                    ] 0% 69.7s133.5 MiB [                    ] 0% 77.0s133.5 MiB [                    ] 0% 83.3s133.5 MiB [                    ] 0% 77.5s133.5 MiB [                    ] 0% 80.8s133.5 MiB [                    ] 0% 66.1s133.5 MiB [                    ] 0% 65.7s133.5 MiB [                    ] 0% 63.4s133.5 MiB [                    ] 0% 57.4s133.5 MiB [                    ] 0% 52.4s133.5 MiB [                    ] 0% 50.7s133.5 MiB [                    ] 0% 52.8s133.5 MiB [                    ] 0% 43.7s133.5 MiB [                    ] 0% 46.5s133.5 MiB [                    ] 0% 50.1s133.5 MiB [                    ] 0% 47.2s133.5 MiB [                    ] 0% 42.4s133.5 MiB [                    ] 0% 42.0s133.5 MiB [                    ] 1% 40.0s133.5 MiB [                    ] 1% 40.1s133.5 MiB [                    ] 1% 38.4s133.5 MiB [                    ] 1% 37.4s133.5 MiB [                    ] 1% 37.8s133.5 MiB [      

In [43]:
!pip install ddgs

In [56]:
promise_vw = """* Ziel: Dekarbonisierung (Kapitel 1)
* Ziel: Kreislaufwirtschaft (Kapitel 2)
* Ziel: Menschen in der Transformation (Kapitel 3)
* Ziel: Vielfalt (Kapitel 4)
* Ziel: Integrität und Compliance (Kapitel 5)
* Ziel: Lieferkette und Menschenrechte (Kapitel 6)
* Ziel: Verbesserung der Leistung in ESG-Ratings und -Rankings des Kapitalmarkts, um die Investitionsfähigkeit zu steigern und Kapitalkosten zu optimieren.
"unser Ziel, ein weltweit führender Anbieter nachhaltiger Mobilität zu sein,"
* Zugang zu 600.000 Ladepunkten in Europa 
* Der CO2-Fußabdruck des gesamten Handelsnetzes soll bis 2030 um mindestens 30 % gesenkt werden.
* Mehr als 850.000 t CO2 wurden seit 2017 durch den AluminiumClosed-Loop bilanziell eingespart. 
* 2023 haben wir 77.090 Werkzeuge bearbeitet, sodass sie für ihren erneuten Einsatz geeignet sind. 
* 14,3 Mio. Weiterbildungsstunden im Volkswagen Konzern 
* 72 Standorte des Volkswagen Konzerns sind nach ISO 45001 zertifiziert.
* 94 % der Führungskräfte haben am Diversity Wins @ Volkswagen Programm teilgenommen.
Unser Ziel ist es, die Natur und die Gesellschaft positiv mitzugestalten.
"klare Ziele, klare Verantwortung" 
"setzen wir uns in der Nachhaltigkeit klare Ziele und Pläne – verbindlich und mit eindeutigen Verantwort lichkeiten."
Unser Ziel ist es, die produktionsbedingten CO2-Emissionen von Pkw und leichten Nutzfahrzeugen bis 2030 um 50,4 Prozent im Vergleich zum Basisjahr 2018 zu reduzieren.
Die CO2-Emissionen in der Nutzungsphase (Scope 3) um 30 Prozent zu reduzieren
mit dazu beizutragen, die Erderwärmung auf 1,5 Grad Celsius zu begrenzen.
* Eine Senkung des durchschnittlichen CO2-Ausstoßes pro Fahrzeug über den gesamten Lebenszyklus um 30 Prozent bis zum Jahr 2030 im Vergleich zu 2018
* Bau von neuen Wind- und Solarparks in mehreren Regionen Europas bis 2025
Diese Veränderungen steuern und begleiten wir nachhaltig sozial im Sinne unserer Kolleginnen und Kollegen.
Beschäftigungssicherung bis 2029 in der Volkswagen AG
Schutz der Menschenrechte
Minimierung umweltbezogener Risiken
Bis zum Jahr 2025 möchte der Volkswagen Konzern die Inbetriebnahme von mehr als 40.000 Schnellladepunkten in China, Nordamerika und Europa ermöglichen.
Wir wollen nachhaltige Mobilität für Generationen ermöglichen.
Umweltentlastung Produktion (UEP)
Dekarbo nisierungsindex (DKI) 
Kreislaufwirtschaft
Bilanzielle CO2-Neutralität wird erreicht, wenn anthropogene CO2-Emissionen weltweit durch die Vermeidung, Reduktion und Kompensation über einen spezifischen Zeitraum ausgeglichen werden.
priorisiert dabei die nachhaltige Wertschöpfung über dem Volumenwachstum.
Eine nachhaltige Lieferkette und umweltverträgliche Transporte sind wichtig, um Verantwortung für Menschenrechte, Umweltschutz und Anti-Korruption umfassend wahrzunehmen.
Verwertung Neben der Verwertung der Fahrzeuge am Ende ihrer Lebensdauer achten wir bereits bei der Entwicklung neuer Fahrzeuge auf die Recyclingfähigkeit der benötigten Werkstoffe, die Verwendung von qualitativ hochwertigem Recyclingmaterial und die Vermeidung von Schadstoffen.
"Demnach erwarten wir, dass der Markt für Elektrofahrzeuge in den nächsten Jahren weiter wachsen wird, weshalb die kosteneffiziente und nachhaltige Produktion von Batteriesystemen und der Ausbau der Ladeinfrastruktur erfolgsentscheidend sein werden."
"Das autonome Fahren wird das Mobilitätserlebnis der Kunden nachhaltig verändern..."
"ein weltweit führender Anbieter nachhaltiger Mobilität zu sein"
Eine zukunftsorientierte Mechatronikplattform soll ... die nachhaltige Wettbewerbsfähigkeit bei Volkswagen bilden.
Unternehmens-DNA | Dekarbonisierung
wir wollen uns zu einem profitablen Experten entlang des gesamten Batterielebenszyklus entwickeln.
Unser vorrangiges Ziel ist es... bis zum Jahr 2030 in bis zu 80 % der Konzernmodelle zum Ein satz kommen soll.
"bis 2030 etwa zur Hälfte"
Bis zum Jahr 2025 sind gemeinsam mit Partnern rund 45.000 Schnellladepunkte (HPC – High-Power-ChargingPunkte) in Europa, China und den USA geplant.
Ladevorgänge sollen dann gezielt so gesteuert werden, dass erneuerbare Energien geladen und Stromnetze entlastet werden können.
Volkswagen will...zu einem aktiven Teil des Energiesystems werden können, um so zu einer dekarbonisierten Mobilität beizutragen.
Decarbonization
Dekarbonisierung, Kreislaufwirtschaft
Wir bekennen uns zum Pariser Klimaschutzabkommen und richten unsere Aktivitäten am 1,5-Grad-Ziel aus. Bis 2050 beabsichtigen wir, ein bilanziell CO2-neutrales Unternehmen zu sein. 
Außerdem haben wir uns zum Ziel gesetzt, eine CO2-Reduktion von Pkw und leichten Nutzfahrzeugen über den gesamten Lebenszyklus von 30 % in 2030 im Vergleich zu 2018 zu erreichen.
den Anteil der Nutzung von erneuerbaren Energien in der Produktentstehung zu erhöhen
den Anteil an kreislauffähigen Materialien in unseren Fahrzeugen zu erhöhen
Das Management des Konzerns soll daher bis 2025 mindestens zu einem Fünftel aus Frauen und mindestens zu einem Viertel aus internationalen Managern bestehen.
Die dafür benötigte Schlüsseltechnologie soll bis 2030 in einem Großteil der plattformbasierten Fahrzeuge integriert sein.
Bis 2030 soll Nordamerika zur dritten Kernregion neben Europa und China werden.
Wir wollen unser konzernweites Angebot an vollelektrischen Modellen massiv ausbauen
Der Anteil batterieelektrischer Fahrzeuge an unseren Verkäufen in den USA und Kanada soll bis 2030 auf 55 % steigen.
nachhaltige Reduzierung von Kosten zur langfristigen Absicherung der Profitabilität
Unser Ziel ist es, im Jahr 2030 in China eine führende Rolle als internationaler Mobilitätsanbieter und als Hersteller von vollvernetzten Fahrzeugen einzunehmen.
"ein starkes, nachhaltiges und sozialverantwortliches Unternehmensumfeld schaffen"
* ...seine Fixkostenstruktur zu optimieren
* ...die Produktivität der Werke zu optimieren
* ...die Kosten der Beschaffung zu optimieren
* ...die Vertriebskosten zu optimieren
* ...das Working-Capital-Management zu optimieren
Operative Umsatzrendite 8,1 % 9 bis 11 %
Investitionsquote im Konzernbereich Automobile ~ 9 %
Cash Conversion Rate im Konzernbereich Automobile > 60 %
Nettoliquidität im Konzernbereich Automobile ~ 10 % der Umsatzerlöse des Konzerns
Kapitalrendite (RoI) im Konzernbereich Automobile > 18 %
Der Dekarbonisierungsindex (DKI) ist ein ESG-relevanter KPI, der vergütungsrelevant für die Mitglieder des Vorstands ist.
"zu einer kontinuierlichen Verbesserung der Nachhaltigkeitsleistung"
Unser Aktionsprogramm, das auf eine Verbesserung unserer Resultate in ESG-Ratings bis zum Jahr 2025 abzielt.
Wir streben an, die negativen Auswirkungen von rechtlichen oder medial ausgetragenen Kontroversen bezüglich des Volkswagen Konzerns auf unsere Rating-Resultate zu reduzieren.
Wir streben an, zukünftig umfassende ESG-Informationsangebote für relevante Stakeholder zu schaffen.
Der Volkswagen Konzern will die Wirkung seines Handelns (Impact) künftig noch umfassender quantitativ messen.
"Die Entwicklung einheitlicher Bewertungsstandards für die Impact-Bewertung sowie die finanzielle Bilanzierung von Nachhaltigkeitswirkungen"
Unternehmens-DNA | Dekarbonisierung | Kreislaufwirtschaft | Menschen in der Transformation | Vielfalt | Integrität und Compliance | Lieferkette und Menschenrechte
"unser Unternehmensziel der bilanziellen CO2-Neutralität bis 2050"
"erfüllen die Kategorie "Clean Transportation" der Green Bond Principles der International Capital Market Association (ICMA), ... sowie im Einklang mit den Zielen der Vereinten Nationen sowie der Europäischen Union für nachhaltige Entwicklung stehen."
"müssen die Belastungen in Bezug auf Lärm, Verkehrsstaus, Unfälle, Verkehrsflächen oder Luftqualität weiter reduziert werden."
zum Klimaschutz beitragen, Lösung von lokalen Umwelt- und Verkehrsproblemen
Minderung der CO2-Emissionen
Städte wollen möglichst wenig Fläche mit Verkehr belegen 
Wichtige Bausteine auf dem Weg zur nachhaltigen Mobilität sind nach wie vor Maßnahmen zur Effizienzsteigerung
"zielt darauf ab, Mobilittslücken im ÖPNV-Angebot zu schließen und zu einer verlässlichen Mobilität beizutragen."
" Als lang jähriges Mitglied im World Business Council for Sustainable Development (WBCSD) hat der Volkswagen Konzern 2023 seine Mitarbeit im "Transport and Mobility Pathway" fortge setzt – einem von neun Aktionsfeldern in der Vision 2050 des WBCSD.
Während der Standzeit des Fahrzeugs sollte das Laden beziehungsweise die Rückspeisung ins Netz mit der Verfügbarkeit von erneuerbarer Energie synchronisiert werden.
Gesellschaftliche Teilhabe verbessern 
Neben ökologischen und ökonomischen Zielen spielen auch soziale Aspekte wie Zugang und Teilhabe an der Gesellschaft eine wichtige Rolle für Mobilitätslösungen.
15 barrierefreie Fahrzeuge einzusetzen (seit Jahresbeginn 2023)
Bis 2026 soll Astypalea nun zu einer smarten und grünen Insel transformiert werden. Der Verkehr auf Astypalea wird auf E-Mobilität umgestellt. Alle Neuwagen, die hier neu zugelassen werden, sind inzwischen reine E-Autos.
2024 soll ... der Solarpark auf 3,5 MW Leistung erweitert und um einen Batteriespeicher ergänzt werden, um ... bis zu 60 % des gesamten Energiebedarfs der Insel mit regenerativer Energie zu decken.
Dekarbonisierung 
Kreislaufwirtschaft
Wir möchten... zielgruppengerecht informieren, aktiv in den Transformationsprozess einbinden, indem wir ihr Feedback und ihre Anregungen für die nachhaltige Entwicklung unseres Unternehmens und der Gesellschaft fordern und fördern.
"unser Ziel ist... seine Akzeptanz und Reputation nachhaltig zu verbessern."
2024 sollen ein systematischer Analyseprozess für Stakeholder implementiert werden.
"Wir wollen...die Anregungen und Empfehlungen unserer Stakeholder künftig noch besser aufnehmen."
Umweltauflagen konsequent einhalten 
Die Einhaltung umweltbezogener Gesetze und Vorgaben an allen unseren Standorten hat für Volkswagen hohe Priorität.
unsere Umweltleistung kontinuierlich zu verbessern und unsere Umweltauswirkungen zu verringern.
Dekarbonisierung und Kreislaufwirtschaft sind die Fokusthemen im Bereich Umweltschutz.
"Unser Umweltleitbild goTOzero"
"Mit diesem Leitbild streben wir danach, Umweltauswirkungen entlang des gesamten Lebenszyklus – von der Rohstoffgewinnung bis zum Lebensende – für alle unsere Produkte und Mobilitäts lösungen zu reduzieren, um Ökosysteme intakt zu halten."
Compliance mit Umweltregulierungen, Standards und freiwilligen Selbstverpflichtungen eine Grundvoraus setzung für unser Handeln.
Wir bekennen uns zum Pariser Klimaschutzabkommen und streben ein 1,5-Grad-Ambitions niveau an. Wir setzen konsequent auf die Elektrifizierung unserer Pro dukte, die Dekarbonisierung unserer gesamten Wertschöp fungskette sowie den Ausbau erneuerbarer Energieerzeugung zur Belieferung unserer Stand orte und Kunden.
Bis spätestens 2050 wollen wir ein bilanziell CO2-neutrales Unternehmen sein. 
Wir reduzieren schädliche Emissionen in Luft, Böden und Gewässer.
Durch den Einsatz von recyceltem Material und nachwachsenden Rohstoffen verringern wir unseren Bedarf an Primärrohstoffen.
Wir verbessern unsere Energie- und Ressourceneffizienz 
Wir etablieren Kreisläufe für Werkstoffe und Wasser. 
Gemeinsam mit unseren Geschäftspartnern verringern wir die Beanspruchung natürlicher Ressourcen entlang unserer Lieferkette.
Wir betreiben effektive UmweltCompliance-Managementsysteme, um Umweltrisiken und -chancen entlang des Lebenswegs unserer Mobilitätslösungen zu identifizieren und zu steuern.
Dekarbonisierung
"die ökologischen Auswirkungen unserer Prozesse, Produkte und Dienstleistungen zu verringern"
"das Umwelt-Compliance-Managementsystem sowie die Umweltleistung kontinuierlich zu verbessern"
Alle Produktions- und Entwicklungsstandorte sollen ihre Umweltmanagementsysteme nach ISO 14001 zertifizieren oder nach dem EcoManagement and Audit Scheme (EMAS) validieren lassen.
"Unsere Umwelt-Compliance-Managementsysteme sollen sicherstellen, dass ökologische Aspekte und Pflichten in unserer Geschäftstätigkeit erkannt und angegemessen berücksichtigt werden."
"Die konzernweite Steuerung des Umweltschutzes erfolgt durch den Konzernsteuerkreis Umwelt und Energie. Dies schließt die Umsetzung der Programme zur Ressourceneffizienz und die Kontrolle der Zielerreichung ein."
Zum Nachweis der Zielerreichung werden jährlich die wesentlichen Umweltkennzahlen offengelegt
Wir wollen das Management von chemischen Substanzen weiter verbessern.
Einsatz gefährlicher Chemikalien reduzieren.
Bis 2025 sollen die produktionsbedingten Umweltbelastungen an allen Standorten zur Fertigung von Pkw und leichten Nutzfahrzeugen im Vergleich zu 2010 um 45 % pro Fahrzeug reduziert werden.
• Energiebedarf pro Fahrzeug: –21,0 % (2022: –14,2 %)
• CO2-Emissionen pro Fahrzeug: –51,0 % (2022: –43,0 %)
• Wasserverbrauch pro Fahrzeug: –24,7 % (2022: –17,4 %)
• Abfall zur Beseitigung pro Fahrzeug: –75,9 % (2022: –68,5 %) 
• VOC-Emissionen pro Fahrzeug: –68,7 % (2022: –66,0 %)
"Verbesserung der Umweltauswirkungen unserer Fabriken ... die UEP verbesserte sich insgesamt deutlich von –37,4 % auf –44,2 %."
Umweltschutz in der Fertigung verankern: Zero Impact Factory Mit der Funktionalbereichsstrategie one.PRODUCTION planen wir die Produktion von morgen. Ressourceneinsatz und Emissionen der Standorte des Volkswagen Konzerns erfordern dabei besondere Aufmerksamkeit. Im Programm "Zero Impact Factory" entwickeln wir konkrete Schritte hin zu einer nachhaltigeren Produktion.
"insbesondere in den Handlungsfeldern Klimaschutz und Energie, Emissionen, Wasser und Abfall."
Strategische Reduzierung von Ressourcen und Emissionen 
Umweltentlastung Produktion (UEP) 
45 % 2010–2025 global lokal 
Zero Impact Factory – ökologische Transformation der Konzernproduktion
Bis 2050 streben wir einen "Zero Impact"-Status für alle Standorte zur Fertigung von Pkw und leichten Nutzfahrzeugen an.
Durch eine Kreislaufführung beziehungsweise die Rückfüh rung von Kühl- und Prozesswasser können der Frischwasser bedarf sowie die Abwassermenge deutlich gesenkt werden. An allen unseren Konzernstandorten fördern wir wasserspa rende Prozesse in der Produktion mit zentralen Vorgaben.
Der spezifische Frischwasser verbrauch für die Fertigung von Pkw und leichten Nutzfahr zeugen ist von 2010 bis 2023 um 24,7 % pro Fahrzeug ge sunken
"Wir haben [...] entsprechende Handlungsfelder definiert, um im Rahmen unseres wirtschaftlichen Handelns unseren Beitrag zum Erreichen dieser Ziele zu leisten."
Im Rahmen des CBD und der 15. Weltnaturkonferenz (CBD COP 15) haben wir im Jahr 2022 unser Biodiversität-Commitment mit seinen sechs Handlungsfeldern weiter konkretisiert.
Wir unterstützen die Initiative Action Agenda for Nature and People des CBD
"lokale Maßnahmen umfassen das Anlegen von Blühwiesen, das Anpflanzen von Bäumen und Sträuchern und das Aufstellen von Nisthilfen für Fledermäuse, Vögel und Insekten."
Um die Biodiversität an den Produktionsstandorten transparent zu machen und schrittweise zu erhöhen
* "Maßnahmen@Web"
* Umgesetzte Maßnahmen
* Standorte mit ISO-14001-Zertifizierung oder EMAS-Validierung: 
  + Volkswagen Konzern: 106 (2023), 105 (2022)
  + bezogen auf Mitarbeiterzahl in %: 99,9 (2023), 98,3 (2022)
* Standorte mit ISO-50001-Zertifizierung:
  + Volkswagen Konzern: 64 (2023), 61 (2022)
  + bezogen auf Endenergieverbrauch der Produktionsstandorte in %: 88,5 (2023), 80,0 (2022)
* Umweltentlastung Produktion (UEP) | GRI 302-5:
  + Veränderung UEP, gesamt in %: –44,2 (2023), –37,4 (2022)
  + Veränderung spezifischer Energiebedarf in %: –21,0 (2023), –14,2 (2022)
  + Veränderung spezifische CO2-Emissionen in %: –51,0 (2023), –43,0 (2022)
  + Veränderung spezifische VOC-Emissionen in %: –68,7 (2023), –66,0 (2022)
  + Veränderung spezifischer Wasserverbrauch in %: –24,7 (2023), –17,3 (2022)
  + Veränderung spezifischer Abfall zur Beseitigung in %: –75,9 (2023), –68,5 (2022)
Bis 2025 wird der zusätzliche jährliche GRC-Regelprozess durch ein standardisiertes IKS ersetzt.
Die Master-Kontroll kataloge werden jährlich auf Aktualität hin geprüft und bei Bedarf angepasst.
Wir werden unsere Beschaffungssysteme erweitern, um Abhängigkeiten bzw. unzureichende Substituierbarkeiten von Lieferanten frühzeitig zu erkennen und diesen entgegenzuwirken.
Dekarbonisierung 
Kreislaufwirtschaft
Bis 2050 beabsichtigen wir, ein bilanziell CO2-neutrales Unternehmen zu sein.
"Aligned with Society and Environment"
Auf dem Weg zum bilanziell CO2-neutralen Unternehmen
* Bis 2050 soll der globale Temperaturanstieg auf deutlich unter 2 Grad Celsius begrenzt werden.
* Bis zum Jahr 2050 will Volkswagen ein bilanziell CO2-neutrales Unternehmen sein.
* Bis 2030 wollen wir den CO2-Fußabdruck unserer Pkw und leichten Nutzfahrzeuge um 30 % (gegenüber 2018) pro Fahrzeug verringern.
Die Umstellung der gesamten Stromversorgung der Werke auf erneuerbare Energien.
"mehr als zwei Drittel (68 %) der zwischen 2023 und 2027 geplanten Investitionen ... in die Zukunftsfelder ... Elektrifizierung."
goTOzero, das für eine bilanziell CO2-neutrale Wirtschaftsweise steht.
Dekarbonisierung |
Dekarbonisierung, Nachhaltigkeitsmanagement
DKI-Ziel erreichen
"Die Energieversorgung wird langfristig auf erneuerbare Energien umgestellt"
"die Zerti fizierung seiner selbst gesetzten Dekarbonisierungs ziele"
"das Risiko durch die erfahrungsbasierte und frühzeitige Identifikation von Schwachstellen im Rahmen der Produktentstehung mit dem Ziel, Fahrzeuganläufe konzernweit hinsichtlich Stückzahl, Qualität und Termingerechtigkeit zu sichern."
Es gibt kein explizites Nachhaltigkeitsversprechen mit einem konkreten Ziel im Text.
Transformation des Unternehmens hin zu einem führenden Anbieter nachhaltiger Mobilität
"stetig neue und verbrauchsgünstige Fahrzeuge und alternative Antriebe entwickelt werden"
Dekarbonisierungsziele
"gezielte Förderung des Aufbaus zusätzlicher Erzeugungskapazitäten für Strom aus erneuerbaren Quellen"
Reduktionsziele sowie Dekarbonisierungsstrategie
Emissionsreduktionsziele
Der Text enthält kein Nachhaltigkeitsversprechen mit einem konkreten Ziel.
Dazu zäh len etwa Maßnahmen für eine effizientere Beleuchtung auf LED-Basis, eine modernisierte Wärme- und Kälte versorgung an den Standorten oder auch optimierte Wasch- und Trocknungsprozesse in der Produktion.
CO2-Neutralität bis 2050
* Das "2°C Scenario" (2DS)
* Das "Beyond 2°C Scenario" (B2DS)
* Bilanzielle CO2-Neutralität des Konzerns bis 2050
"Um die UN-Klimaziele zu erreichen"
Bis 2035 noch über die Hälfte des Marktanteils behalten.
die Energieversorgung in der gesamten Wertschöpfungskette und in allen Lebensphasen eines Fahrzeugs sukzessive auf regenerative Energie umstellt.
den durchschnittlichen globalen Temperaturanstieg bis 2100 auf 2 Grad Celsius zu begrenzen
den durchschnittlichen globalen Temperaturanstieg auf 1,75 Grad Celsius begrenzen könnten. 
emissionsbedingte Risiken weitestgehend in strategische Entscheidungsprozesse integrieren und Reduktionspfade der CO2-Flottencompliance optimieren
Umstellung auf erneuerbare Energiequellen entlang der gesamten Wertschöpfungskette
Kompensation der nicht vermeidbaren Emissionen, z.B. durch Investition in Wiederaufforstung
Rund 90 % der angestrebten Dekarbonisierung des Volkswagen Konzerns sind aus heutiger Sicht über die Elektrifizierung der Flotte und den Umstieg auf regenerativ erzeugten Strom rea lisierbar.
bilanziell CO2-neutrale Mobilität
Die Auslieferung des voll-elektrischen Kompaktwagens ID.3 erfolgt bilanziell CO2-neutral.
* Dekarbonisierung entlang des Lebenszyklus 
 * Emissionsfreie Nutzungsphase dank erneuerbarer Energien
* Erhöhung des Anteils erneuerbarer Energien im Energiemix
"bilanziell nahezu CO2-neutrale Mobilität"
Bis 2025 sollen so in verschiedenen Regionen Europas neue Wind- und Solarparks entstehen.
Bis zum Jahr 2025 sollen alle Projekte zusammen rund 7 TWh an zusätzlichem Ökostrom erzeugen.
Alle Lieferanten (Neuvergaben) von HV-Batterien werden bereits heute vertraglich dazu verpflichtet, zertifizierten Strom aus erneuerbaren Quellen in ihrer Produktion einzusetzen. Damit sinkt der CO2-Ausstoß in der Batteriefertigung.
"Bei neuen Fahrzeugprojekten macht der Volkswagen Konzern die CO2-Emissionen in Zukunft zu einem technischen Merkmal für relevante Bauteile. Das bedeutet: Wir geben den Lieferanten verbindliche CO2-Ziele vor, deren Einhaltung sie zu jeder Zeit nachweisen können müssen."
Ein Beispiel betrifft die neue Mechatronik-Plattform SSP. So sind die Batterien der SSP-Plattform mit einem CO2-Grenzwert versehen. Um diese Werte erreichen zu können, müssen die Lieferanten Maßnahmen in ihren eigenen Produktionsprozessen und Vorlieferketten umsetzen, zum Beispiel den Einsatz von erneuerbaren Energien.
"so soll die CO2-Bilanz der ID.-Familie in den nächsten Jahren um rund zwei Tonnen je Fahrzeug verbessert werden."
"100 % erneuerbare Energien bis 2030."
Leitwerk Salzgitter mit geplantem Anlauf im Jahr 2025, Valencia in Spanien (2026) und St. Thomas in Kanada (2027).
Alle Standorte werden konsequent mit Strom aus erneuerbaren Quellen betrieben. 
 In Spanien soll ein angrenzender Foto voltaik-Park auf 250 Hektar Fläche bis zu 30 % des Strom bedarfs abdecken.
* Ein wichtiger Schritt auf dem Weg zu nachhaltig erzeugten eigenen Batteriezellen ist der PowerCo durch erfolgreiche Tests des neuen Produktionsverfahrens Dry Coating (Trocken beschichtung) gelungen, das rund 30 % Energie in der Zellfer tigung, 15 % Fabrikfläche und Fertigungskosten in Millionen höhe einsparen kann.
Die PowerCo-Zellfabriken sind künftig darauf ausgelegt, die Materialnutzung innerhalb der Produktion durch fertigungs nahes Recycling der Produktionsüberschüsse (scrap sowie end-of-line) zu maximieren.
Die Partner streben an, bis 2030 Produktionskapazitäten für Kathodenmaterial und Vorläufermaterialien aufzubauen, die einer jährlichen Zellproduktion von 160 GWh entsprechen.
"damit soll der Aufbau regionaler, nachhaltiger und transparenter Wertschöpfungsketten für Batterien beschleunigt und die EU bei der Erreichung der Ziele des Green Deals unterstützt werden."
bis 2030 etwa 900 Arbeitsplätze entstehen sollen
Transporte vermieden und Emissionen reduziert werden – unter anderem auch mit den Mitteln der Digitalisierung.
nahezu vollständige CO2-Vermeidung durch den Einsatz von Ökostrom im Schienenverkehr in Deutschland und weiteren Ländern
Bis zum Jahr 2026 werden alle unsere neuen Schiffsnamen nach Vögeln benannt sein. Unsere Flotte soll bis spätestens 2040 kohlendioxidneutral sein, und wir streben danach, dass dies auch für die gesamte Transportkette gilt."
nahezu kein Methan entweichen kann
Die Konzernlogistik möchte dadurch die CO2-Emissionen noch weiter reduzieren.
"die Dekarbonisierung in anderen Industrien vorantreiben."
Volkswagen will den Ausstoß von Treibhausgasemissionen in der Produktion bis 2030 absolut um 50,4 % gegenüber 2018 reduzieren.
Der Volkswagen Konzern hat sich das Ziel gesetzt, von 2018 bis 2030 Energieeffizienzmaßnahmen umzusetzen, die in Summe 4,9 Mio. MWh Energie an den Fertigungsstandorten einsparen.
Bis 2023 konnten bereits 7.822 Maßnahmen in Höhe von 3,0 Mio. MWh um gesetzt werden.
Die Stilllegung der Steinkohleblöcke im Heizkraftwerk Wolfsburg-West ist für den 31. März 2024 geplant.
Volkswagen geht davon aus, dass die ursprünglich avisierte jährliche Einsparung von 1,5 Mio. t CO2 ab dem zweiten Quartal 2024 jährlich realisiert werden kann. 
Die Versorgung der Werke mit Strom aus erneuerbaren Energien kommt weiter voran.
Bis 2030 ist dieselbe Zielmarke für alle weltweiten Standorte geplant.
So haben wir uns das Ziel gesetzt, bis 2030 1,2 Mio. MWh Strom aus erneuerbaren Energien selbst beziehungsweise in unmittelbarer Nähe der Fertigungsstandorte zu erzeugen.
Bislang werden 56 Fertigungsstandorte des Konzerns mit externem Strom aus 100 % erneuerbaren Energien versorgt. 56,3 % des gesamten weltweiten Stromverbrauchs der Konzernproduktion (inklusive China) wurde 2023 durch Strom aus erneuerbaren Energien gedeckt.
Wir haben bereits heute 8 Fertigungsstandorte bilanziell CO2-neutral.
Schnellladeinfrastruktur für Pkw ausbauen
3 Standorte in China: Versorgung mit 100 % erneuerbarem bzw. bilanziell CO2-neutralem Strom.
Zusammen mit anderen Original Equipment Manufacturers (OEMs) hat Volkswagen bereits 2017 IONITY gegründet – eine gemeinsame Gesellschaft, mit der bis Ende 2023 europa weit 2.800 Schnellladestationen an Fernstraßen errichtet wurden. Bis 2025 will das Unternehmen im Verbund mit Part nern rund 18.000 öffentliche Schnellladepunkte in Europa betreiben – fünfmal so viele wie heute und rund ein Drittel des für 2025 prognostizierten Gesamtbedarfs auf dem Kontinent.
• In Nordamerika soll die Ladeinfrastruktur von Electrify America bis 2025 auf mehr als 8.000 Standorte erweitert werden. 
• In China plant Volkswagen über das Joint Venture CAMS insgesamt 17.000 Schnellladepunkte bis 2025.
Bis 2025 sind gemeinsam mit Partnern insgesamt mehr als 40.000 Schnellladepunkte in Europa, China und in den USA geplant.
Bis 2025 will Volkswagen circa 400 Mio. € für das Gesamtprogramm in Europa aufwenden.
EUROPA 18.000 Schnellladepunkte Schnellladepunkte bis 2025 
NORDAMERIKA 8.000 Schnellladepunkte 
CHINA 17.000 Schnellladepunkte
Wir haben als freiwillige Maßnahme bereits 2022 beschlossen, eine Reihe von Elektrofahrzeugen des Konzerns bilanziell CO2-neutral an unsere Kunden in Europa zu übergeben.
Solange wir CO2-Emissionen nicht überall vermeiden können und er neuerbare Energien nicht überall einsetzbar sind, geschieht dies über freiwillige Kompensation der verbleibenden Treibhausgasemissionen aus der Lieferkette, Produktion und Logistik.
"Im Jahr 2023 belief sich dieser für den Konzern auf rund 8,45 Mio. t CO2."
Dekarbonisierung des Händlernetzes (im Rahmen des Projekts "goTOzero retail")
- Der CO2-Fußabdruck des Handelsnetzes soll bis 2030 um mindestens 30 % gesenkt werden.
- Der CO2-Fußabdruck des Handelsnetzes soll bis 2040 um mindestens 55 % gesenkt werden.
- Der CO2-Fußabdruck des Handelsnetzes soll bis 2050 um mindestens 75 % gesenkt werden.
"Im letzten Schritt werden nicht vermeidbare CO2-Emissionen kompensiert."
Ab 2024 sollen sukzessive alle 50 wesentlichen Handelsmärkte an der Zertifizierung teilnehmen.
ab 2024 sollen alle Märkte und Partnerbetriebe ein Energie- und Ressourcenberatungskonzept erhalten
Für neu errichtete und bestehende Gebäude strebt der Volkswagen Konzern eine höchstmögliche Energieeffizienz an.
Maßnahmen in diesem Bereich sollten sowohl skaliert werden, als auch die Zusätzlichkeit und Dauerhaftigkeit atmosphärischer Kohlenstoffbindung gewährleisten können.
"die entwickelt und finanziert zertifizierte Klimaschutzprojekte, die dem anerkannten Ausgleich von CO2Emissionen dienen."
"die Zusätzlichkeit, Genauigkeit und Permanenz der Emissionsreduktionen, "
Lösungen für die Dekarbonisierung zu entwickeln.
Dekarbonisierung vorantreiben:
EU zu Vorreiter für Klimaschutz machen: Engagement in der CEO Alliance Volkswagen ist Mitglied in der "CEO Alliance for Europe's Recovery, Reform and Resilience", einer paneuropäischen und sektorübergreifenden Allianz großer Konzerne mit Sitz in Europa, die sich ausdrücklich zum Green Deal der EU und zu den damit verbundenen Klimaschutzzielen bekennen.
Die CEO Alliance unterstützt das Ziel, die EU zur weltweit führenden Region im Klimaschutz zu machen
Verpflichtung zu CO2-neutralen Unternehmensgebäuden bis 2030
"rasche Maßnahmen zur Dekarbonisierung von Mobilität und Verkehr", 
"Transformation des Gebäudesektors"
"zügige Dekarbonisierung des Energiesystems in der EU"
Bis zum Jahr 2050 will der Volkswagen Konzern ein bilanziell CO2-neutrales Unternehmen sein.
Um 50,4 % sollen die produktionsbedingten CO2-Emissionen bis 2030 sinken. Der Konzern hat sich das Ziel gesetzt, die produktionsbe dingten CO2-Emissionen seiner Pkw und leichten Nutzfahr zeuge bis 2030 um 50,4 % zu senken – verglichen mit dem Basisjahr 2018.
"mit seiner Zielsetzung für die Produktionsphase (Scope 1 und 2) ... die Bedingungen zur Begrenzung der Erdwärmung auf 1,5 Grad Celsius erfüllt."
* Reduzierung der CO2-Emissionen in der Nutzungsphase um 30 % (Scope 3)
* Reduzierung der CO2-Emissionen pro Fahrzeug über den gesamten Lebenszyklus um 30 % bis 2030 im Vergleich zu 2018
Scania verpflichtet sich, seine absoluten Scope-1- und Scope-2-Treibhausgasemissionen bis 2025 um 50 % gegenüber dem Basisjahr 2015 zu reduzieren. 
Das Unternehmen kündigte 2022 außerdem an, seine Lieferkette bis 2030 weitestgehend zu dekarbonisieren.
Die Scope-3-Treibhausgasemissionen aus der Nutzung der verkauften Fahrzeuge sollen bei Scania bis 2025 um 20 % pro Fahrzeugkilometer sinken.
Verglichen mit 2019 will das Unternehmen seine Scope-1- und Scope-2-Emissionen bis 2030 um 70 % senken, seine Scope-3-Emissionen um 28 %.
Für das Jahr 2024 plant Volkswagen, die für die Basisjahre berichteten CO2-Werte anzupassen.
Verwendung unternehmensspezifischer Verschnittsraten für Stahl- und Aluminiumbauteile in der Produktion anstatt generischer Daten (seit 2022 [Aluminium] bzw. 2023 [Stahl])
CO2-neutrale Übergabe von Elektrofahrzeugen.
Um die Wirksamkeit unserer Maßnahmen zur Reduzierung der CO2-Emissionen unserer Fahrzeuge zu messen...
"gegenüber einem gesetzlichen Zielwert von 122 g CO2/km (WLTP)."
Eine Zielverschärfung erfolgt ab 2025: Demnach hat die EU-Kommission das Ziel einer CO2-Reduktion von 15 % gegenüber 2021 festgelegt, was für unsere EU-Pkw-Neuwagenflotte einem CO2-Zielwert von unter 100 g CO2/km entspricht. Für das Jahr 2030 wurde eine Absenkung von 55 % festgelegt, dies entspricht einem CO2-Zielwert von unter 50 g CO2/km.
Wir gehen davon aus, dass unsere EU-Pkw-Neuwagenflotte das Ziel für 2025 erfüllen und das Ziel für das Jahr 2030 übererfüllen kann. 
Für das Jahr 2035 gilt ein CO2-Reduktionsziel von 100 % für Pkw.
Eine Zielverschärfung erfolgt ab 2025: Demnach schreibt die EU-Kommission eine CO2-Reduktion von 15 % gegenüber 2021 vor, was für unsere EU-Neuwagen flotte für leichte Nutzfahrzeuge einem CO2-Zielwert von unter 180 g CO2/km entspricht.
* Für das Jahr 2030 wurde eine Absenkung von 50 % festgelegt, dies entspricht einem CO2-Zielwert von unter 105 g CO2/km.
* Wir gehen davon aus, dass unsere EU-Neuwagenflotte für leichte Nutzfahrzeuge das Ziel für 2025 erfüllen und das Ziel für das Jahr 2030 übererfüllen kann.
* Für das Jahr 2035 gilt ein CO2-Reduktionsziel von 100 % für leichte Nutzfahrzeuge.
GHG CO2-Wert für das Modelljahr 2023 liegt im Durchschnitt bei 133 g CO2/km, gegenüber einem gesetzlichen Zielwert von 122 g CO2/km.
Für das Jahr 2025 gehen wir in den USA von einem CO2-Zielwert von circa 110 g CO2/km aus und rechnen damit, dass wir dieses Ziel erreichen.
Für 2030 wollen wir den Elektrofahrzeug-Anteil unserer Neuwagenflotte auf deutlich über 50 % erhöhen.
"erreichen können. Wir haben uns ambitionierte Ziele gesetzt: Bis 2050 wollen wir klimaneutral sein."
"Das Programm und die damit verbunden Maßnahmen sollen Fortschritte bei der Reduzierung von Treibhausgasemissionen ermöglichen."
Eine Weiterentwicklung der Methodik mithilfe internationaler generischer Bezugsdaten wird für die Emissionen der Kategorie "Commuting" im Berichtsjahr 2024 umgesetzt.
Wir gestalten mit unseren taxonomie konformen Investitionen in Entwicklungsleistungen und Sachanlagen bereits heute die Zukunft im Sinne der EU-Taxonomie ökologisch nachhaltig.
Klimaneutralität bis zum Jahr 2050 erreichen.
"Sie zielt darauf ab, die Finanzierung der Transformation zu einer nachhaltigen Wirtschaft zu unterstützen,"
• Klimaschutz 
• Anpassung an den Klimawandel 
• Nachhaltige Nutzung und Schutz von Wasser- und Meeresressourcen 
• Übergang zu einer Kreislaufwirtschaft 
• Vermeidung und Verminderung der Umweltverschmutzung 
• Schutz und Wiederherstellung der Biodiversität und der Ökosysteme.
• Erbringung eines substanziellen Beitrags zu einem der Um weltziele 
• Einhaltung der für die jeweilige Wirtschaftstätigkeit defi nierten Do-No-Significant-Harm(DNSH)-Kriterien
• Einhaltung von Rahmenwerken zum Mindestschutz (Mini mum Safeguards)
Bis 2050 beabsichtigen wir, ein bilanziell CO2-neutrales Unternehmen zu sein.
Vermeidung und Verringerung der Umweltverschmutzung 
Schutz und Wiederherstellung der Biodiversität und der Ökosysteme
Transformation von Volkswagen zu einem Anbieter nachhaltiger Mobilität
kein konkrete Ziel genannt
"Umweltziel: Klimaschutz"
Herstellung, Reparatur, Wartung, Nachrüstung, Umnutzung und Aufrüstung von CO2-armen Fahrzeugen, Schienenfahrzeugen und Schiffen.
Herstellung von Technologien, die auf eine erhebliche Verringerung der Treibhausgasemissionen in anderen Wirtschaftssectoren abzielen
* Verringerung von Treibhausgasemissionen 
* Vermeidung von Treibhausgasemissionen 
* Abbau von Treibhausgasemissionen
* Dekarbonisierung
Bis zum 31. Dezember 2025 gilt dies darüber hinaus für Pkw und leichte Nutzfahrzeuge mit CO2-Emissionen von weniger als 50 g/km gemäß WLTP – darunter fällt der größte Teil unserer Plug-in-Hybride.
soll zukünftig auf weitere Anwendungsfälle ausgedehnt werden
Die Herstellung von Anlagen zur Verwendung von Wasserstoff, wie sie für eine wasserstoffbasierte Energie- und Rohstoffversorgung benötigt werden, leistet einen substanziellen Beitrag zum Klimaschutz.
erhebliche Verringerung der Treibhausgasemissionen
Rüsten wir durch sogenannte "Retrofits" die maritime Bestandsflotte technisch so um, dass Einsparungen von CO2-Emissionen ermöglicht werden.
"Einsparungen an Lebenszyklus-THG-Emissionen"
Verringerung von Treibhausgasemissionen
* "Herausgabe von Anlagen für die Erzeugung und Verwendung von Wasserstoff"
* "Herstellung anderer CO2-armer Technologien"
"Erhaltung der Wasserqualität, Vermeidung von Wasserstress und Umweltverträglichkeitsprüfung"
"Volkswagen definiert in seinen Umweltgrundsätzen, im Weißbuch Gesamtfabrik sowie in der goTOzero-Strategie Vorgaben zur Kreislaufwirtschaft."
"Vermeidung und Verminderung der Umweltverschmutzung" 
"darf eine Wirtschaftstätigkeit, im Vergleich zur Lage vor Beginn der Tätigkeit, nicht zu einem erheblichen Anstieg der Schadstoffemissionen in Luft, Wasser oder Boden führen."
"Vorgaben und Prozesse [...] vorsehen, dass SVHC zu vermeiden und zu ersetzen sind."
Schutz und Wiederherstellung der Biodiversität und der Ökosysteme
"Abhilfemaßnahmen einzuleiten und auf ihre Wirksamkeit zu prüfen", 
"Das Einhalten der darin definierten vertraglich verbindlichen Anforderungen wird von uns mithilfe eines Nachhaltigkeitsratings bei relevanten Lieferanten überprüft."
besonders hohe Risiken in unserer Lieferkette im Zusammenhang mit Menschenrechtsverletzungen sowie Umwelt erkennen und angemessen zu adressieren.
Substanzieller Beitrag zum Klimaschutz
Herstellung von CO2-armen Verkehrstechnologien 
davon taxonomiekonforme BEV
Herstellung von Anlagen für die Erzeugung und Verwendung von Wasserstoff 
Herstellung anderer CO2-armer Technologien
Einhaltung DNSH-Kriterien 
Einhaltung Mindestschutz 
Taxonomiekonforme Investitionsausgaben
Substanzieller Beitrag zum Klimaschutz
Einhaltung DNSH-Kriterien 
Einhaltung Mindestschutz 
Taxonomiekonforme Betriebsausgaben
Herstellung von CO2-armen Verkehrstechnologien
Herstellung von Anlagen für die Erzeugung und Verwendung von Wasserstoff 
Herstellung anderer CO2-armer Technologien
"Die Wirtschaftstätigkeit 3.3 "Herstellung von CO2-armen Verkehrstechnologien" des Umweltziels Klimaschutz."
"unsere CO2-Emissionen deutlich zu senken"
Wir wollen unsere Anstrengungen für einen Übergang zu einer kreislauforientierten und ressourcenschonenden Wirtschaftsweise künftig noch weiter intensivieren.
Unser Beitrag zu einer kreislauffähigen Wirtschaft Im Rahmen der Konzerninitiative 6 verfolgen wir im Bereich Kreislaufwirtschaft mehrere Handlungsstränge. Dazu zählt etwa der verstärkte Einsatz von kreislauffähigen Materialien, Sekundärmaterialien und nachhaltig nachwachsenden Rohstoffen in unseren Fahrzeugen.
Umwelt goTOzero
"setzt sich ... das Ziel, seine Ressourceneffizienz weiter zu verbessern sowie Ansätze zu Wieder verwertung und Recycling in den Bereichen Werkstoffe, Energie und Wasser zu fördern." 
"eine Fabrik ohne negative Auswirkungen auf die Umwelt."
* Unseren Ressourcenverbrauch zu minimieren 
* Den Energieverbrauch zu reduzieren
Fahrzeuge einer langen Nutzungsdauer
Unsere Vorgehensweise bei der Abfallbeseitigung in der Produktion zielt darauf ab, unsere Abfallmenge zu senken und nicht vermeidbare Abfälle hochwertig zu verwerten.
• die Vermeidung der Abfallentstehung 
• Priorisierung der Verwertung von Abfällen und Reduzierung der Abfälle zur Beseitigung
"ein gemeinsames Verständnis des Qualitätsanspruchs an Entsorgungsdienstleistungen zu erreichen, Audits quali tätssicher durchzuführen und die Ergebnisse somit auch für andere Original Equipment Manufacturer und Zulieferer nutzbar zu machen."
Maßnahmen entlang des gesamten Lebenszyklus
Der Einsatz von zirkulären Materialien auf Fahrzeugebene, schlüsselt diesen nach verschiedenen Fahrzeugprojekten auf.
Wir verstärken die Nutzung von Materialkreisläufen in unseren Produktionsprozessen.
Pkw und leichte Nutzfahrzeuge müssen an ihrem Lebensende zu 85 % recycelbar und zu 95 % verwertbar sein
Unter dem Dach der Initiative "Zero Impact Factory" verstärken wir mit dem Projekt "Zero Plastic Waste" unsere Anstrengungen, Kunststoffabfälle zu vermeiden.
Kreislauffähigkeit der Fahrzeuge wird bei deren Entwicklung berücksichtigt, um eine spätere Wiederverwendung von Materialien zu ermöglichen.
• Für viele Bauteile wird der Einsatz von Rezyklaten priorisiert, wenn sie die gleichen Qualitätsansprüche erfüllen wie Primärmaterialien und über die Laufzeit in ausreichender Menge verfügbar sind.
• Alle Bauteile aus Kunststoff werden nach international gültigen ISO-Normen gekennzeichnet, um sie später identifizieren und sortenrein trennen zu können. 
• In der Umweltnorm Fahrzeug bestehen Designempfehlungen, die es ermöglichen, Materialien nach Lebenszyklusende der Fahrzeuge besser voneinander zu trennen.
ein möglichst hoher Anteil wiederverwerteter Materialien
Sitztextilien werden zum Teil aus bis zu 100 % recyceltem PET gefertigt
"reduziert diese Auswirkungen deutlich und hilft uns, unseren CO2-Fußabdruck zu senken."
"den Kreislauf für den Konzern flächendeckend zu schließen."
"Ziel ist die industrialisierte Rückgewinnung wertvoller Rohmaterialien wie Lithium, Nickel, Mangan und Kobalt im geschlossenen Kreislauf sowie von Aluminium, Kupfer und Kunststoff."
Die Anlage ist zunächst darauf ausgelegt, im Pilotbetrieb bis zu 3.600 Batteriesysteme im Jahr zu recyceln.
Mit dem zurückgewonnenen Material kann perspektivisch die Versorgung der Zellfertigung bei Volkswagen unterstützt werden.
Unsere Orientierung an hoher Qualität mit einer geringen Reparaturbedürftigkeit zielt auf eine hohe Langlebigkeit unserer Fahrzeuge in der Nutzungsphase ab und ist damit ein wichtiger Beitrag zur Ressourceneffizienz.
Altteilen werden industriell aufbereitet mit dem Ziel einer möglichen Wiederverwendung in anderen Fahrzeugen des Konzerns.
Stoffkreisläufe geschlossen und die Umweltlasten im Vergleich zu Primärmaterial reduziert werden.
Reduzierung der Treibhausgasemissionen um rund 97 % gegenüber den Primärrohstoffen 
eine Reduzierung der Treibhausgasemissionen in Höhe von knapp 27 Tonnen pro Kilogramm SekundärEdelmetall im Vergleich zu Primär-Edelmetallen
Audi verfolgt das Ziel, sich als nachhaltige Premiummarke zu positionieren.
Audi Original Gebrauchtteile ("Sustain able Parts") sollen ab 2024 schrittweise in den Märkten der EU verfügbar sein.
bis zu 95 % der Energie gegenüber der Verwendung von Primäraluminium eingespart werden
Mehr als 850.000 t CO2 wurden seit 2017 durch den Aluminium-Closed-Loop bilanziell eingespart.
Reduzierung des Energiebedarfs um 3.250 MWh pro Jahr
Reduzierung des CO2-Ausstoßes um über 1.400 t pro Jahr
Langfristig soll die Menge auf bis zu 80 t Späne pro Tag steigen.
jährliche Einsparungen von 2.500 t CO2
wertvolle Materialien auf diese Weise dauerhaft zurückzugewinnen und somit zu mehr Nachhaltigkeit und einer höheren Versorgungssicherheit beizutragen.
"zielgerichtete Dekarbonisierung von Produkten und Geschäftsprozessen"
Wir verpflichten uns zu:
dem rechtlich gültigen und zu garantierenden nationalen Lohnminimum für alle Beschäftigten
der Wahrung der Rechte auf Privatsphäre, persönliche Sicherheit und Meinungsfreiheit
den Rechten indigener Völker 
zur Verhinderung grausamer, unmenschlicher oder erniedrigender Behandlung
Unser Ziel ist es, das Spannungsfeld zwischen den unterschied lichen nationalen Rahmenbedingungen und dem Interesse an einer größtmöglichen Verwirklichung des Vereinigungs rechts zu überbrücken.
"Unser Ziel bei Volkswagen ist eine erfolgreiche und sozial verantwortliche Transformation der Mitarbeiter in die neue Epoche nachhaltiger Mobilität."
"Die Personalstrategie setzt innovative Akzente: Das Mitarbeitererleben (Employee Experience) wird systematisch verbessert, die Teams werden als wichtigste Einheiten der Unternehmensorganisation gestärkt."
Wir wollen das Mitarbeitererleben systematisch verbessern.
"Der eMotion Room ist Teil eines eintägigen Schulungsprogramms (eMotionDay), an dem bis 2025 in Wolfsburg die gesamte Belegschaft der Produktion teilnehmen soll."
"Ziel ist es, die Umsetzung der von den Organisationseinheiten abgeleiteten Maßnahmen nachhaltig sicherzustellen."
Dekarbonisierung
"Wir tragen Verantwortung für Umwelt und Gesellschaft",
"bis 2029"
Gemäß unserer Sozialcharta entsprechen die Vergütungen und Nebenleistungen für unsere Mitarbeiter mindestens dem jeweils rechtlich gültigen und zu garantierenden Minimum des jeweiligen Landes.
ab 2023
• Ziels von 35 %
"Ziel ist, die durch schnittlichen Qualifizierungsstunden pro Mitarbeiter im Volkswagen Konzern – bezogen auf die aktive Belegschaft1 – bis 2030 um 35 % auf 30,0 Stunden pro Jahr zu erhöhen."
"Die Zielgröße für das Berichtsjahr waren 24,0 Stunden."
Klima- und Umweltschutz
Ziel wert des Volkswagen Konzerns für das Jahr 2025 beträgt 89,1 von 100 möglichen Indexpunkten. Zwischenziel für 2023 von 88,8 Indexpunkten
sichere und gesunde Arbeitsumgebung gewährleisten
"Indem wir die höchsten Standards im Bereich Arbeits- und Gesundheitsschutz anstreben..."
Alle Vorgesetzten und Beschäftigten sollen informiert sowie qualifiziert sein und sich sicherheitsgerecht verhalten. Ein sicheres und gesundes Arbeitsumfeld zu gewährleisten, ist insbesondere in der Transformation ein wesentlicher Bestandteil nachhaltiger Unternehmensführung
langfristige Strategien und Maßnahmen entwickeln und umsetzen, die darauf abzielen, die Sicherheit und Gesundheit der Beschäftigten zu gewährleisten und gleichzeitig ökologische und soziale Aspekte zu berücksichtigen
"Konkret bietet...zu fördern."
Bis 2026 sind alle Produktionsstandorte mit mehr als 1.000 eigenen Beschäftigten nach der ISO 45001 zu zertifizieren.
eine durchgehende, digitale Prozesskette zu schaffen, für Beschäftigte mit gesundheitlichen Problemen einen frühestmöglichen Therapiebeginn zu gewährleisten und die Gesundheitskompetenz der Be schäftigten zu steigern.
"Wir wollen bis 2025 die Anzahl der Arbeitsunfälle pro eine Million geleisteter Arbeitsstunden um 50 Prozent reduzieren. Im Berichtsjahr lag diese Quote bei 3,6 (Vorjahr: 4,1). Wir haben unsere Maßnahmen verstärkt, um unsere Mitarbeiter im Umgang mit Gefahrstoffen und in der Handhabung von Maschinen und Anlagen zu schulen sowie die Sicherheit unserer Fahrzeuge weiter zu erhöhen. Ziel ist es, die Zahl der schwerwiegenden Unfälle auf null zu reduzieren."
Die Zertifizierung für die sechs Standorte der Volkswagen AG ist bis zum Jahr 2024 geplant.
dass sich alle Beschäftigten in ihrer Individualität gesehen fühlen und ihr volles Potenzial entfalten können.
"ab 2023 ... Einführung eines ESG-Faktors im Jahresbonus ... Zielerreichung der indexrelevanten Frauenanteile im Management"
Inhouse-Kompetenz auszubauen, um Alltagsdiskriminie rung und -rassismus konsequent entgegenzuwirken.
Wir streben auf allen Management-Ebenen an, Maßnahmen und Programme mit Zielen zu hinterlegen und die Diversität im Unternehmen weiter zu fördern.
einen prozentualen Frauenanteil im Management vorzugeben und Ziele für den Internationalisierungsgrad im Top-Management festzulegen.
"Zukunftsfähige Unternehmen sind für uns zentraler Teil der Lösung von morgen. Unser Ziel ist es, dass bis 2030 mindestens 80% unseres Portfolios aus nachhaltigen Unternehmen bestehen soll. Dazu entwickeln wir unsere bestehenden Fonds kontinuierlich weiter und bringen neue Produkte auf den Markt."
Bis 2025 wollen wir den Frauenanteil im Management auf 20,2 % steigern. Den Internationalisierungsgrad im Top-Management ... möchten wir bis 2025 auf 25,0 % erhöhen.
Für das Jahr 2023 war eine Steigerung dieses Index auf 142 geplant.
"Ziele für die Erhöhung des Frauenanteils in der Volkswagen AG 
Wir haben zudem für die Volkswagen AG gemäß § 76 Abs. 4 des deutschen Aktiengesetzes Ziele im Hinblick auf Frauenanteile im Management formuliert."
"hat sich die Volkswagen AG ... einen Frauenanteil von 16,5 % für die erste Füh rungsebene ... und von 23,4 % für die zweite Führungsebene ... innerhalb der aktiven Belegschaft zum Ziel gesetzt."
Programme und Angebote Mit verschiedenen Programmen verfolgen wir das Ziel, die Frauenanteile in den Führungsebenen des Konzerns zu erhöhen und für mehr Chancengerechtigkeit und Gleichberechtigung in der Karriereentwicklung zu sorgen.
"um die nachhaltige Erhöhung der Frauenanteile im Management zu erreichen"
Volkswagen beteiligt sich am Programm "Target Gender Equality", das vom UN Global Compact aufgelegt wurde und eine verstärkte Gleichstellung der Geschlechter zum Ziel hat.
Ansätze zur Förderung von Vielfalt und Inklusion ausbauen
Bis zum 31. Dezember 2023 wurden konzern- und weltweit 94 % der Führungskräfte von Gesellschaften mit über 1.000 Beschäftigten geschult.
94 % unserer Führungskräfte haben am Diversity Wins @ Volkswagen Programm teilgenommen.
Das geschäftsbereichsübergreifende "Women Career Network" setzt sich für gleiche Entwicklungschancen von Frauen und mehr Diversität in Führungspositionen im Volkswagen Konzern ein.
Kein Nachhaltigkeitsversprechen mit konkretem Ziel gefunden.
"Ein hohes Maß an Flexibilität bietet...bis zum 31. Dezember 2023"
Frauenanteil im Management Zielwert Frauenanteil im Management in % 17,9 17,0 
Internationalisierung im Top-Management Zielwert Internationalisierung im Top-Management in % 23,2 22,3
Wir richten unsere Regelwerke, Prozesse und unsere Unternehmenskultur darauf aus... dass sich jeder Einzelne an Gesetze, Regelungen und Selbstverpflichtungen hält.
Unternehmens-DNA | Dekarbonisierung | Kreislaufwirtschaft | Menschen in der Transformation | Vielfalt | Integrität und Compliance | Lieferkette und Menschenrechte
"Ziel des Programms ist auch, die Integritätskultur weiterzuentwickeln."
Implementierung des ICMS bis 2025
* Dekarbonisierung
"Wir wollen erreichen, dass integres Verhalten im Geschäftsalltag selbstverständlich ist."
Der Volkswagen Konzern vertritt eine Null-Toleranz-Politik gegenüber aktiver und passiver Korruption.
Risiken für Gesetzesverstöße und das Missachten ethischer Standards früh erkennen, 
riskante Geschäftspartner meiden, 
Maßnahmen zur Risikominimierung festlegen und umsetzen.
Alle Beschäftigten im Konzern sind verpflichtet, zweijährlich ein CoC-Training mit Abschlusstest zu absolvieren.
Sie bestätigen, sich an die Vorgaben zu halten... Teil der Zertifizierung ist auch, dass die Teilnehmer die Meldepflicht bei Schweren Regelverstößen kennen und eventuelle Interessenkonflikte offenlegen.
"so sollen Marken und Gesellschaften ... risikobasiert, zielgruppenspezifisch und mit vorab festgelegten Kerninhalten qualitativ gleichwertig zu trainieren."
Wir schulen Beschäftigte und Führungskräfte über Produkt-Compliance und richten Stellen ein, an die Mitarbeiter ihre Fragen richten können.
Wir wollen die personenbezogenen Daten von aktuellen und ehemaligen Beschäftigten sowie von Kunden, Lieferanten, Vertragspartnern und anderen Betroffenen schützen.
Seit 2021 bieten wir ein verpflichtendes webbasiertes Datenschutztraining für alle Führungskräfte sowie Konzern- und Markenvorstände der Volkswagen AG an.
Alle zwei Jahre werden Mitarbeiter zu datenschutzrelevanten Themen sensibilisiert, Ziel ist die Erhöhung des Bewusstseins für den Umgang mit personenbezogenen Daten.
"Zudem hat die Volkswagen AG ein Datenschutzmanagementsystem (DSMS) sowie eine Datenschutzorganisation etabliert."
Hier sind die Nachhaltigkeitsversprechen mit konkreten Zielen:

* Eine gültige Qualifizierung zum Code of Conduct für 98,9% der Beschäftigten in der Volkswagen AG (2023)
* Eine gültige Qualifizierung zum Thema Anti-Korruption für 98,6% der Beschäftigten in der Volkswagen AG (2023)
"Wir lehnen jegliche Nutzung von Kinder-, Zwangs- und Pflichtarbeit sowie jegliche Form von moderner Sklaverei und Menschenhandel ab."
Das für das erste Jahr der Geltung des Lieferkettensorgfalts pflichtengesetzes (LkSG) auf Menschenrechte bezogene strategische Ziel des Volkswagen Konzerns ist die voll umfängliche und bestmögliche Erfüllung der sich aus dem Gesetz ergebenden Pflichten.
Es war für das Geschäftsjahr 2023 zunächst unser Anspruch, die gesetzlichen Anforderungen fristgerecht und lückenlos umzusetzen.
"In den kommenden Jahren werden wir unser initiales Risiko management zur Verteidigung menschenrechtlicher und umweltbezogener Schutzgüter kontinuierlich überprüfen, ver bessern und um weitere strategische Ziele und Schutzgüter, auch über das LkSG hinaus, erweitern."
Der Volkswagen Konzern wendet sich entschieden gegen Zwangs- und Kinderarbeit im Zusammenhang mit seinen Geschäftsaktivitäten.
Dekarbonisierung 
Kreislaufwirtschaft
Vermeidung menschenrechtlicher und umweltbezogener Risiken
Unser Ziel ist es, Lieferketten verantwortungsvoll zu gestalten und Risiken bestmöglich zu reduzieren.
"Klima neutralität"
Die Umsetzung von Projekten und Partnerschaften im Bereich Kreislaufwirtschaft 
Die Reduzierung der CO2-Emissionen in der Lieferkette
menschliche, soziale oder ökologische Risiken entlang der Lieferkette des Volkswagen Konzerns zu vermeiden beziehungsweise zu minimieren.
• Nachhaltigkeitsleistung der Lieferanten kontinuierlich zu verbessern.
Batterie zellen: eine möglichst vollständige Offenlegung der Liefer ketten von Kobalt, Nickel, Lithium und natürlichem Graphit. 
Bei neuen Fahrzeugprojekten: die CO2-Emissionen in Zukunft zu einem technischen Merkmal für relevante Bauteile zu machen.
Wir zeigen Möglichkeiten zur kontinuierlichen Verbesserung auf.
Lieferanten mit einem A- oder B-Rating erfüllen unsere Anforderungen in ausreichendem Umfang und sind somit vergabefähig. Erfüllt ein Lieferant unsere Anforderun gen zur Einhaltung von Nachhaltigkeitsstandards nicht (C-Rating), so ist er grundsätzlich nicht vergabefähig.
"und Risiken abgeschwächt werden sollen."
Bis zum Ende des Berichtszeitraums haben insgesamt 14.953 direkte Lieferanten einen SAQ im Rahmen des S-Ratings abgeschlossen, bei 9.357 Lieferanten wurde eine Verbesserung der Nachhaltigkeitsleistung erzielt.
Gemäß dem Code of Conduct für Geschäfts partner ist ein zertifiziertes und/oder validiertes Umweltma nagementsystem für alle Lieferanten mit Produktionsstandort und Standortgröße ab 100 Mitarbeitern Pflicht. Basierend auf dem Umsatz haben über 80 % unserer Lieferanten im Scope diese Anforderung erfüllt.
Ab einer Bewertung von unter 80 % werden Verbesserungsmaßnahmen in einem Plan festgehalten, dessen Umsetzung mit dem Lieferanten vereinbart und nachverfolgt wird.
* eine breitere Abdeckung der Lieferkette zu ermöglichen 
* den operativen Aufwand für Lieferanten niedrig zu halten
4.639 Lieferanten verfügen über ein A-Rating.
2023 wurden weltweit 7.791 Lieferanten zu Nachhaltigkeit geschult.
3 Die MAN Energy Solutions SE hat nach eigener Methodik eine eigene Risikoanalyse bei Zulieferern durchgeführt.
7.791 Lieferanten wurden im Berichtszeitraum entsprechend geschult.
engagieren wir uns auch für den Schutz derjenigen Gruppen, die entlang unserer Lieferkette einem hohen Risiko poten zieller Menschenrechtsverletzungen ausgesetzt sein können.
Es gibt kein explizites Nachhaltigkeitsversprechen mit einem konkreten Ziel im gegebenen Text.
Der Code of Conduct fordert Lieferanten auf, ihren Beschäftigten einen ange messenen Lohn zu zahlen. Dieser soll existenzsichernd sein.
Verringerung des Scopes
"Ziel ist es, dass alle unsere Lieferanten befähigt sind, unsere Nachhaltigkeitsanforderungen effektiver und umfassender in die tiefere Lieferkette zu tragen."
Im Hinblick auf einen verantwortungsvollen Rohstoffbezug setzt der Volkswagen Konzern die fünf Schritte der "Due Diligence Guidance for Responsible Business Conduct" der OECD und die Anforderungen der "OECD Due Diligence Guidance for Responsible Supply Chains of Minerals from Conflict-Affected and High-Risk Areas" um.
Wir überprüfen die Wirksamkeit der Präventions- und Abhilfemaßnahmen.
Wir möchten menschenrechtliche Risiken auch in der vor gelagerten Lieferkette und über unsere vertraglichen Beziehungen hinaus adressieren.
Entwicklung von standardisierten Werkzeugen zur Risikobewertung 
Einführung von Standards für verantwortungsvolle Rohstofflieferketten in Bezug auf Menschenrechte, Umwelt und Compliance
Für die Batterierohstoffe Kobalt, Lithium, Nickel und natürliches Graphit ist es unser Ziel, vom Abbau in der Mine bis zur Herstellung des fertigen Produktes Transparenz in der Lieferkette herzustellen.
"das soziale Wohlergehen der Menschen vor Ort zu verbessern", "die Gesundheits- und Sicherheitsbedingungen ... zu verbessern"
Die Initiative "Responsible Lithium Partnership" setzt sich für eine nachhaltige Lithiumgewinnung in der Salar de Atacama in Chile ein.
Verringerung des Scopes.
* "direkte Lieferanten in Scope mit zertifiziertem Umweltmanagementsystem gemäß ISO-14001- oder EMAS-Validierung": Ziel: 80% (2023) bzw. 85% (2022)
* "Anteil der neuen Lieferanten, die anhand von Sozial- und Umweltkriterien bewertet wurden": Ziel: 26% (2023) bzw. 12% (2022)
Es gibt kein Nachhaltigkeitsversprechen mit einem konkreten Ziel im gegebenen Text.
* Dekarbonisierung
"""

In [55]:
import requests
from bs4 import BeautifulSoup
from ddgs import DDGS
import time

# Initialisierung des Ollama-Clients (siehe Doku von htw-ollama-py)
OLLAMA_MODEL = "phi4:14b"

# Nachhaltigkeitsversprechen
claims = [
    "VW verspricht CO2-Neutralität bis 2050",
    "VW will den Anteil von E-Autos bis 2030 auf 70 Prozent erhöhen",
    "VW reduziert Emissionen in der Lieferkette um 30 Prozent bis 2030"
]

def suche_artikel(query, max_results=5):
    with DDGS() as ddgs:
        return list(ddgs.text(query, max_results=max_results))

def scrape_text(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        r = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(r.text, "html.parser")
        paragraphs = soup.find_all('p')
        return '\n'.join(p.get_text() for p in paragraphs)
    except Exception as e:
        print(f"Fehler beim Abrufen von {url}: {e}")
        return ""

def analysiere_mit_ollama(text, claim):
    company = "Volkswagen"
    prompt = f"""
        "{company}" hat folgendes Nachhaltigkeitsversprechen abgegeben:

        "{claim}"

        Hier ist ein Ausschnitt eines Artikels zur Überprüfung:

        {text[:2000]}

        Basierend auf diesem Artikel:
        - Wurde das Versprechen eingehalten? (ja / teilweise / nein)
        - falls eine Strategie zur Erreichung des Ziels vorgestellt wurde, ist dies realistisch/umsetzbar?
        - Gib eine kurze Begründung.
        - Gib das Ergebnis im folgenden JSON-Format zurück:

        {{
        "status": "ja / teilweise / nein",
        "reason": "Begründung in einem Satz"
    }}
"""

    res = OllamaApi.completion(prompt, model=OLLAMA_MODEL)
    print("results: ", res.get("result"))
    try:
        return res.get("result")
    except Exception as e:
        print(f"❌ JSON-Fehler bei Auswertung: {e}")
        return None

def bewerte_claim(claim, company):
    print(f"\n🔍 Suche nach Artikeln zum Thema: {claim}")
    #ergebnisse = suche_artikel(f"intitle:{company} AND intitle:{claim} after:2023" ) #site:spiegel.de OR site:zeit.de OR site:sueddeutsche.de")
    ergebnisse = suche_artikel(f"(({claim} OR '{company} Nachhaltigkeitsziele' OR '{company} Greenwashing') AND (site:nachhaltigkeit-wirtschaft.de)) after:2024-01-01") #Klammern um die site-Bedingungen, damit sie sich auf alle Suchbegriffe beziehen

    for eintrag in ergebnisse:
        url = eintrag.get("href", "")
        if not url:
            continue
        print(f"📄 Artikel gefunden: {url}")
        text = scrape_text(url)
        if not text:
            continue
        bewertung = analysiere_mit_ollama(text, claim)
        print(f"Bewertung durch Ollama:\n{bewertung}\n")
        time.sleep(3)  # Vermeidung von Rate Limits

# Hauptdurchlauf
for c in claims:
    company = "Volkswagen"
    bewerte_claim(c, company)



🔍 Suche nach Artikeln zum Thema: VW verspricht CO2-Neutralität bis 2050
📄 Artikel gefunden: https://nachhaltigkeit-wirtschaft.de/glossar/smart-grid/
results:  Basierend auf dem Artikel wurden keine spezifischen Informationen über Volkswagen oder dessen Strategie zur Erreichung der CO2-Neutralität bis 2050 bereitgestellt. Der Artikel konzentriert sich stattdessen allgemein auf die Vorteile und Rolle von Smart Grids im Kontext nachhaltiger Energiesysteme.

Hier ist das Ergebnis im angeforderten JSON-Format:

```json
{
    "status": "nein",
    "reason": "Der Artikel enthält keine spezifischen Informationen über Volkswagen oder dessen Maßnahmen zur Erreichung der CO2-Neutralität."
}
```

**Zusätzliche Bemerkungen:**

1. **Verbindung zu Smart Grids**: Während Smart Grids erheblich zum Übergang zu nachhaltigerer Energie beitragen können, deckt der Artikel nicht die spezifischen Pläne von Volkswagen ab.

2. **Realisierbarkeit einer potenziellen Strategie**: Da keine Details zu VWs Ansatz vo

KeyboardInterrupt: 